In [1]:
from langchain.embeddings import CacheBackedEmbeddings



In [ ]:
%pip install --upgrade --quiet  langchain-openai faiss-cpu



In [ ]:
from langchain_ollama import OllamaEmbeddings

underlying_embeddings = OllamaEmbeddings(
    #model="nomic-embed-text:latest",
    model="llama3.2:latest",
    base_url="http://localhost:11434",  # Replace with your Ollama base URL
)


In [17]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter



store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

In [ ]:
list(store.yield_keys())



In [19]:
raw_documents = TextLoader("../../00-example_data/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
%%time
db = FAISS.from_documents(documents, cached_embedder)